In [ ]:
import pandas as pd
import numpy as np
import json
import gzip

In [ ]:
with gzip.GzipFile('data.json.gz','r') as fp:
    data = json.loads(fp.read())

In [ ]:
#data['teams'][0]

In [ ]:
data['players'][0].keys()

In [ ]:
def getdiff(x):
    res = {}
    try:
        for k in x:
            ok = 'opp' + k[0].upper() + k[1:] 
            if ok in x:
                if k[:2] in ['fg','ft','tp'] and (len(k) < 3 or k[2] != 'a'):
                    res[k + '%'] = 100*(x[k]/max(1,x[k[:2]+'a' + k[2:]])) #- x[ok]/max(1,x[ok[:5]+'a' + ok[5:]]))
                elif k == 'pts':
                    res[k] = x[k] - x[ok]
                else:
                    res[k] = x[k] #- x[ok]
    except:
        print(k)
        raise
    return res
data2 = sum([[getdiff(_) for _ in team['stats'] if not _['playoffs']] for team in data['teams']],[])
d2 = sum([[_['won'] for _ in team['seasons']] for team in data['teams']],[])
for i in range(len(d2)):
    data2[i]['wins'] = d2[i]
getdiff(data['teams'][0]['stats'][1])


In [ ]:
df = pd.DataFrame(data2)

In [ ]:
import statsmodels.api as sm

In [ ]:
X = df[[_ for _ in df.columns if _ not in ['wins','pts']]]
y = df['pts']/50
X = (X-X.mean())/X.std()
X = sm.add_constant(X)
clf = sm.OLS(y,X)
clf_res = clf.fit()
clf_res.summary()

In [ ]:
coeff = clf_res.params
coeff.iloc[coeff.abs().argsort()]

In [ ]:
data['players'][0]['ratings']

In [ ]:
data['players'][0]['stats'][0]

In [ ]:
skillset = set()
for p in data['players']:
    for s in p['ratings']:
        skillset = skillset.union(set(s['skills']))

In [ ]:
data3 = {}
for p in data['players']:
    pdata = {}
    #pdata['pid'] = p['pid']
    for s in p['stats']:
        if s['playoffs']:
            continue
        pdata[s['season']] = {'team':s['tid']}
        pdata[s['season']]['min'] = s['min']
        pdata[s['season']]['per'] = s['per']
        pdata[s['season']]['ows'] = s['ows']
        pdata[s['season']]['dws'] = s['dws']
        #pdata[s['season']]['ewa'] = s['ewa']
        #pdata[s['season']]['drtg'] = s['drtg']
        #pdata[s['season']]['ortg'] = s['ortg']

    for r in p['ratings']:
        ud = pdata.get(r['season'],{})
        ud.update(r)
        pdata[r['season']] = ud
    for s in pdata:
        for c in ['G','C','F']:
            pdata[s][c+'_min'] = float(c in pdata[s]['pos'])
        for skill in skillset:
            pdata[s][skill+'_smin'] = float(skill in pdata[s]['skills'])
        del pdata[s]['fuzz']
        del pdata[s]['pos']
        del pdata[s]['pot']
        del pdata[s]['season']
        del pdata[s]['skills']
    data3[p['pid']] = pdata

In [ ]:
data_n = list(data3[0][2020].keys())
data4=[]
for p in data3:
    for s in data3[p]:
        data4.append([s]+[data3[p][s].get(_,np.nan) for _ in data_n])

In [ ]:
df = pd.DataFrame(data4,columns=['season'] + data_n)
df = df.dropna()
df

In [ ]:
for col in df.columns:
    if col in ['season','team','min']:
        continue
    df[col] = df[col]*df['min']
df_r = df.groupby(['season','team']).sum()#/df['min']

In [ ]:
df_r2 = df_r.reset_index()
for col in df_r2:
    if col in ['season','team','min']:
        continue
    df_r2[col] = df_r2[col]/np.maximum(1,df_r2['min'])

In [ ]:
#df_pr = df_r2[df_r2 >0].dropna(how='all')
df_r2 = df_r2[df_r2.season < df_r2.season.max()]

In [ ]:
#data['teams'][0]#['seasons'][0]

In [ ]:
d2 = sum([[_['won'] for _ in team['seasons']] for team in data['teams']],[])
d3 = sum([[team['tid'] for _ in team['seasons']] for team in data['teams']],[])
d4 = sum([[_['season'] for _ in team['seasons']] for team in data['teams']],[])

for i in range(len(d2)):
    data2[i]['wins'] = d2[i]
    data2[i]['team'] = d3[i]
    data2[i]['season'] = d4[i]

In [ ]:
df = pd.DataFrame(data2)

In [ ]:
df = df[df.season < df.season.max()]

In [ ]:
df

In [ ]:
df_r2['season'] = df_r2['season'].astype(int)
df_r2['team'] = df_r2['team'].astype(int)
dt = df_r2.set_index(['season','team']).join(df.set_index(['season','team']),rsuffix='_r')

In [ ]:
from sklearn import preprocessing

In [ ]:
X = dt[[_ for _ in dt.columns if 'min' not in _ and _ not in ['per','dws','ows','min','err','pred','wins','pts','fg%', 'fga','fgAtRim%', 'fgaAtRim', 'fgLowPost%', 'fgaLowPost', 'fgMidRange%',  'fgaMidRange', 'tp%', 'tpa', 'ft%', 'fta', 'orb', 'drb_r', 'ast', 'tov',  'stl', 'blk', 'pf']]]
y = dt['pts']/50
if False:
    ss = preprocessing.PolynomialFeatures()
    X2 = np.nan_to_num(ss.fit_transform(X))
    X2 = (X2-X2.mean())/X2.std()

    X2 = pd.DataFrame(X2,columns=ss.get_feature_names(X.columns))
    X2.index = X.index
    X = X2
else:
    #X = (X-X.mean())/X.std()
    X = sm.add_constant(X)


clf = sm.OLS(y,X)
clf_res = clf.fit()

legit_vals = []
for v in clf_res.pvalues.iteritems():
    if v[1] < 0.05:
        legit_vals.append(v[0])
#clf_res.summary()
if False:
    clf = sm.RLM(y,X[legit_vals])
    clf_res = clf.fit()
clf_res.summary()

In [ ]:
import matplotlib.pyplot as plt
import scipy.stats

In [ ]:
plt.style.use('fivethirtyeight')
plt.style.use('seaborn-white')
plt.scatter(dt['ovr'],dt['pts'],s=3,alpha=0.8)
plt.xlabel('overall (min weighted)')
plt.ylabel('point margin')
plt.title('r squared: {:.3f}'.format(scipy.stats.pearsonr(dt['ovr']/50,dt['pts'])[0]**2))
plt.grid(True)

In [ ]:
plt.style.use('fivethirtyeight')
plt.style.use('seaborn-white')
plt.scatter(dt['pts']/50,clf_res.predict(),s=3,alpha=0.8)
plt.xlabel('actual margin')
plt.ylabel('predicted margin')
plt.title('r squared: {:.3f}'.format(scipy.stats.pearsonr(dt['pts']/50,clf_res.predict())[0]**2))
plt.grid(True)

In [ ]:
dt['pred'] = clf_res.predict()
dt['err'] = (dt['pts']/50- clf_res.predict())#/np.maximum(1,abs(dt['pts']/50))
dt['err'] = (dt['err']-dt['err'].mean())/dt['err'].std()

In [ ]:
dt[dt.err > 3.4][['ovr','pts','err','pred']]

In [ ]:
data['teams'][21]['name']

In [ ]:
dt.iloc[:,:-21]

In [ ]:
coeff = clf_res.params
coeff.iloc[coeff.abs().argsort()]

In [ ]:
pd.set_option("display.precision", 2)
clf_res.params

In [ ]:
X.mean()